<a href="https://colab.research.google.com/github/lseidy/IA_learning/blob/master/Decision_Tree_Fruits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import pandas as pd
import seaborn as sns
import gdown
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
import plotly.express as ex
import numpy as np

### Carregamento dos dados

Treino: https://drive.google.com/file/d/1L-C3liPU63KUdQdW7Dv2pyqeY8ctNf_N/view?usp=drive_link

Teste: https://drive.google.com/file/d/1OVnNfPEjp_q_HcXJ_M43gC0c0SwSkW3v/view?usp=drive_link

In [3]:
!gdown '1L-C3liPU63KUdQdW7Dv2pyqeY8ctNf_N'
!gdown '1OVnNfPEjp_q_HcXJ_M43gC0c0SwSkW3v'

Downloading...
From: https://drive.google.com/uc?id=1L-C3liPU63KUdQdW7Dv2pyqeY8ctNf_N
To: /content/fruit_train.csv
100% 1.20k/1.20k [00:00<00:00, 4.71MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OVnNfPEjp_q_HcXJ_M43gC0c0SwSkW3v
To: /content/fruit_test.csv
100% 313/313 [00:00<00:00, 1.24MB/s]


In [24]:
treino = pd.read_csv('fruit_train.csv')
teste = pd.read_csv('fruit_test.csv', delimiter=';')

In [5]:
treino.head(10)

,mass,width,height,color_score,fruit_name
0,192,8.4,7.3,0.55,apple
1,180,8.0,6.8,0.59,apple
2,86,6.2,4.7,0.80,mandarin
3,84,6.0,4.6,0.79,mandarin
4,80,5.8,4.3,0.77,mandarin
5,80,5.9,4.3,0.81,mandarin
6,178,7.1,7.8,0.92,apple
7,172,7.4,7.0,0.89,apple
8,172,7.1,7.6,0.92,apple
9,154,7.0,7.1,0.88,apple


In [6]:
teste.head(10)

,mass,width,height,color_score,fruit_name
0,176,7.4,7.2,0.60,apple
1,76,5.8,4.0,0.81,mandarin
2,166,6.9,7.3,0.93,apple
3,168,7.5,7.6,0.73,apple
4,170,7.6,7.9,0.88,apple
5,362,9.6,9.2,0.74,orange
6,164,7.2,7.0,0.80,orange
7,150,7.1,7.9,0.75,orange
8,194,7.2,10.3,0.70,lemon
9,132,5.8,8.7,0.73,lemon


In [7]:
treino.describe()

,mass,width,height,color_score
count,48.000000,48.000000,48.000000,48.000000
mean,161.166667,7.106250,7.679167,0.763125
std,51.531165,0.762249,1.324717,0.074553
min,80.000000,5.800000,4.300000,0.550000
25%,140.000000,6.650000,7.200000,0.720000
50%,157.000000,7.250000,7.550000,0.750000
75%,178.500000,7.500000,8.125000,0.810000
max,356.000000,9.200000,10.500000,0.920000


In [8]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   mass         48 non-null     int64  
 1   width        48 non-null     float64
 2   height       48 non-null     float64
 3   color_score  48 non-null     float64
 4   fruit_name   48 non-null     object 
dtypes: float64(3), int64(1), object(1)
memory usage: 2.0+ KB


In [9]:
graph = ex.scatter_matrix(treino,dimensions=['mass', 'width', 'height', 'color_score'], color= 'fruit_name')
graph.show()

Separei a coluna do nome das frutas, pois é ela que queremos prever

In [55]:
labels = treino.iloc[:,4]
labels_teste = teste.iloc[:,4]

In [57]:
labels_teste

,fruit_name
0,apple
1,mandarin
2,apple
3,apple
4,apple
5,orange
6,orange
7,orange
8,lemon
9,lemon


Utilizei Label encoder para transformar as variaveis categorias em numéricas

In [58]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels_teste = label_encoder.fit_transform(labels_teste)

In [59]:
labels = labels.reshape(-1,1)
labels_teste = labels_teste.reshape(-1,1)
labels

array([[0],
       [0],
       [2],
       [2],
       [2],
       [2],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [60]:
fruit_treino = treino.iloc[:, [0,1,2,3]]
fruit_teste = teste.iloc[:, [0,1,2,3]]

In [26]:
fruit_treino

,mass,width,height,color_score
0,192,8.4,7.3,0.55
1,180,8.0,6.8,0.59
2,86,6.2,4.7,0.80
3,84,6.0,4.6,0.79
4,80,5.8,4.3,0.77
5,80,5.9,4.3,0.81
6,178,7.1,7.8,0.92
7,172,7.4,7.0,0.89
8,172,7.1,7.6,0.92
9,154,7.0,7.1,0.88


Utilizei também o Onehot para padronizar os labels e StandardScaler pra padrozinar o resttante dos dados

In [61]:
onehot_fruits = OneHotEncoder()
labels = onehot_fruits.fit_transform(labels).toarray()
labels_teste = onehot_fruits.fit_transform(labels_teste).toarray()
labels

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],


In [62]:
scaler = StandardScaler()
fruit_treino = scaler.fit_transform(fruit_treino)
fruit_teste = scaler.fit_transform(fruit_teste)
fruit_treino

array([[ 0.60467527,  1.71524057, -0.28925351, -2.88896613],
       [ 0.36934219,  1.18492465, -0.67068671, -2.34675548],
       [-1.47410024, -1.20149702, -2.27270615,  0.49985045],
       [-1.51332242, -1.46665498, -2.34899279,  0.36429778],
       [-1.59176678, -1.73181294, -2.57785271,  0.09319246],
       [-1.59176678, -1.59923396, -2.57785271,  0.63540311],
       [ 0.33012001, -0.00828619,  0.09217969,  2.1264824 ],
       [ 0.21245347,  0.38945076, -0.51811343,  1.71982441],
       [ 0.21245347, -0.00828619, -0.06039359,  2.1264824 ],
       [-0.14054614, -0.14086517, -0.44182679,  1.58427175],
       [ 0.05556475,  0.25687178,  0.01589305, -0.85567619],
       [-0.17976832,  0.65460872, -0.28925351, -0.99122885],
       [-0.10132396,  0.7871877 , -0.44182679, -0.99122885],
       [-0.10132396,  0.65460872, -0.13668023, -1.26233418],
       [ 0.01634257,  0.52202974, -0.44182679,  0.90650843],
       [ 0.01634257,  0.38945076, -0.36554015,  1.17761376],
       [-0.0228796 ,  0.

Criei a Arvore de Decisao e fiz o treinamento

In [63]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(fruit_treino,labels)

DecisionTreeClassifier()

In [66]:
prediction = decision_tree.predict(fruit_teste)
prediction

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])

Oberservando

In [78]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

print(accuracy_score(labels_teste, prediction))
print(classification_report(labels_teste,prediction))

0.9090909090909091
              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         1
           3       0.75      1.00      0.86         3

   micro avg       0.91      0.91      0.91        11
   macro avg       0.94      0.94      0.93        11
weighted avg       0.93      0.91      0.91        11
 samples avg       0.91      0.91      0.91        11



### Juntando datasets

In [77]:
dataset = np.concatenate((treino,teste))
dataset

array([[192, 8.4, 7.3, 0.55, 'apple'],
       [180, 8.0, 6.8, 0.59, 'apple'],
       [86, 6.2, 4.7, 0.8, 'mandarin'],
       [84, 6.0, 4.6, 0.79, 'mandarin'],
       [80, 5.8, 4.3, 0.77, 'mandarin'],
       [80, 5.9, 4.3, 0.81, 'mandarin'],
       [178, 7.1, 7.8, 0.92, 'apple'],
       [172, 7.4, 7.0, 0.89, 'apple'],
       [172, 7.1, 7.6, 0.92, 'apple'],
       [154, 7.0, 7.1, 0.88, 'apple'],
       [164, 7.3, 7.7, 0.7, 'apple'],
       [152, 7.6, 7.3, 0.69, 'apple'],
       [156, 7.7, 7.1, 0.69, 'apple'],
       [156, 7.6, 7.5, 0.67, 'apple'],
       [162, 7.5, 7.1, 0.83, 'apple'],
       [162, 7.4, 7.2, 0.85, 'apple'],
       [160, 7.5, 7.5, 0.86, 'apple'],
       [156, 7.4, 7.4, 0.84, 'apple'],
       [140, 7.3, 7.1, 0.87, 'apple'],
       [342, 9.0, 9.4, 0.75, 'orange'],
       [356, 9.2, 9.2, 0.75, 'orange'],
       [204, 7.5, 9.2, 0.77, 'orange'],
       [140, 6.7, 7.1, 0.72, 'orange'],
       [160, 7.0, 7.4, 0.81, 'orange'],
       [158, 7.1, 7.5, 0.79, 'orange'],
       [210, 